In [22]:
# from huggingface_hub import snapshot_download
# local_dir = snapshot_download("unsloth/Qwen2.5-VL-3B-Instruct-bnb-4bit", local_dir="./md/unsloth")

In [23]:
# from huggingface_hub import snapshot_download

# train_dataset = snapshot_download('PKU-Alignment/Align-Anything', repo_type="dataset", allow_patterns=["text-image-to-text/train/train-00000-of-00015.parquet"], local_dir='./ds2')

In [ ]:
# from huggingface_hub import snapshot_download

# train_dataset = snapshot_download('trl-lib/rlaif-v', repo_type="dataset", allow_patterns=["data/train-00004-of-00014.parquet"], local_dir='./ds3')

/home/intisar/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [00:48<00:00, 48.95s/it]


In [1]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096
dtype = torch.bfloat16
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/media/intisar/dataset1/visual_categorization/Sigbath/md/unsloth",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.7.7: Fast Qwen2 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA RTX A4000. Num GPUs = 1. Max memory: 15.61 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [ ]:
# # @title Alignment Handbook utils
# import os
# import re
# from typing import List, Literal, Optional

# from datasets import DatasetDict, concatenate_datasets, load_dataset, load_from_disk
# from datasets.builder import DatasetGenerationError


# DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|im_start|>user\n' + message['content'] + <|im_end|> + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|im_start|>system\n' + message['content'] + <|im_end|> + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|im_start|>assistant\n'  + message['content'] + <|im_end|> + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|im_start|>assistant' }}\n{% endif %}\n{% endfor %}"


# def apply_chat_template(
#     example,
#     tokenizer,
#     task: Literal["sft", "generation", "rm", "dpo"] = "sft",
#     assistant_prefix="<|im_start|>assistant\n",
# ):
#     def _strip_prefix(s, pattern):
#         # Use re.escape to escape any special characters in the pattern
#         return re.sub(f"^{re.escape(pattern)}", "", s)

#     if task in ["sft", "generation"]:
#         messages = example["messages"]
#         # We add an empty system message if there is none
#         if messages[0]["role"] != "system":
#             messages.insert(0, {"role": "system", "content": ""})
#         example["text"] = tokenizer.apply_chat_template(
#             messages,
#             tokenize=False,
#             add_generation_prompt=True if task == "generation" else False,
#         )
#     elif task == "rm":
#         if all(k in example.keys() for k in ("chosen", "rejected")):
#             chosen_messages = example["chosen"]
#             rejected_messages = example["rejected"]
#             # We add an empty system message if there is none
#             if chosen_messages[0]["role"] != "system":
#                 chosen_messages.insert(0, {"role": "system", "content": ""})
#             if rejected_messages[0]["role"] != "system":
#                 rejected_messages.insert(0, {"role": "system", "content": ""})
#             example["text_chosen"] = tokenizer.apply_chat_template(
#                 chosen_messages, tokenize=False
#             )
#             example["text_rejected"] = tokenizer.apply_chat_template(
#                 rejected_messages, tokenize=False
#             )
#         else:
#             raise ValueError(
#                 f"Could not format example as dialogue for `rm` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
#             )
#     elif task == "dpo":
#         if all(k in example.keys() for k in ("chosen", "rejected")):
#             from PIL import Image
#             import requests
#             from io import BytesIO
#             import base64

#             img = BytesIO(example["image"]["bytes"])
#             image_data = base64.b64encode(img.getvalue()).decode('utf-8')

#             prompt_message = [
#                 {
#                     "role": "user",
#                     "content": [
#                         {
#                             "type": "image",
#                             "image": f"data:image;base64,{image_data}" , 
#                         },
#                         {
#                             "type": "text",
#                             "text": example["question"],
#                         },
#                     ],
#                 },

#             ]

#             chosen_messages = [{
#                     "role": "assistant",
#                     "content": [
#                         {
#                             "type": "text",
#                             "text": example["chosen"],  # Chosen response
#                         },
#                     ],
#                 }]
            
#             rejected_messages = [{
#                     "role": "assistant",
#                     "content": [
#                         {
#                             "type": "text",
#                             "text": example["rejected"],  # Rejected response
#                         },
#                     ],
#                 }]

#             example["text_chosen"] = tokenizer.apply_chat_template(
#                 chosen_messages, tokenize=False
#             )
#             example["text_rejected"] = tokenizer.apply_chat_template(
#                 rejected_messages, tokenize=False
#             )
#             example["text_prompt"] = tokenizer.apply_chat_template(
#                 prompt_message, tokenize=False, add_generation_prompt=True
#             )
#             example["text_chosen"] = _strip_prefix(
#                 example["text_chosen"], assistant_prefix
#             )
#             example["text_rejected"] = _strip_prefix(
#                 example["text_rejected"], assistant_prefix
#             )

#             from qwen_vl_utils import process_vision_info
#             image_inputs, video_inputs = process_vision_info(prompt_message)
#             example["images"] = image_inputs


#         else:
#             raise ValueError(
#                 f"Could not format example as dialogue for `dpo` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
#             )
#     else:
#         raise ValueError(
#             f"Task {task} not supported, please ensure that the provided task is one of {['sft', 'generation', 'rm', 'dpo']}"
#         )
#     return example


# def get_datasets(
#     data_config: dict,
#     name: Optional[str] = None,
#     splits: List[str] = ["train", "test"],
#     shuffle: bool = True,
# ) -> DatasetDict:


#     if type(data_config) is dict:
#         dataset_mixer = data_config
#     else:
#         raise ValueError(f"Data config {data_config} not recognized.")

#     raw_datasets = mix_datasets(dataset_mixer, name, splits=splits, shuffle=shuffle)
#     return raw_datasets


# def mix_datasets(
#     dataset_mixer: dict, name:str, splits: Optional[List[str]] = None, shuffle=True
# ) -> DatasetDict:

#     raw_datasets = DatasetDict()
#     raw_train_datasets = []
#     raw_val_datasets = []
#     fracs = []
#     for ds, frac in dataset_mixer.items():
#         fracs.append(frac)
#         for split in splits:
#             try:
#                 dataset = load_dataset(ds, split=split, name=name)
#             except DatasetGenerationError:
#                 dataset = load_from_disk(os.path.join(ds, split))

#             if "train" in split:
#                 raw_train_datasets.append(dataset)
#             elif "test" in split:
#                 raw_val_datasets.append(dataset)
#             else:
#                 raise ValueError(
#                     f"Split type {split} not recognized as one of test or train."
#                 )

#     if any(frac < 0 for frac in fracs):
#         raise ValueError("Dataset fractions cannot be negative.")

#     if len(raw_train_datasets) > 0:
#         train_subsets = []
#         for dataset, frac in zip(raw_train_datasets, fracs):
#             train_subset = dataset.select(range(int(frac * len(dataset))))
#             train_subsets.append(train_subset)
#         if shuffle:
#             raw_datasets["train"] = concatenate_datasets(train_subsets).shuffle(seed=42)
#         else:
#             raw_datasets["train"] = concatenate_datasets(train_subsets)
#     # No subsampling for test datasets to enable fair comparison across models
#     if len(raw_val_datasets) > 0:
#         if shuffle:
#             raw_datasets["test"] = concatenate_datasets(raw_val_datasets).shuffle(
#                 seed=42
#             )
#         else:
#             raw_datasets["test"] = concatenate_datasets(raw_val_datasets)

#     if len(raw_datasets) == 0:
#         raise ValueError(
#             f"Dataset {dataset_mixer} not recognized with split {split}. Check the dataset has been correctly formatted."
#         )

#     return raw_datasets

# raw_datasets = get_datasets(
#     {"/media/intisar/dataset1/visual_categorization/Sigbath/ds" : 0.005},
#     splits=["train"],
# )

# column_names = list(raw_datasets["train"].features)

# raw_datasets = raw_datasets.map(
#     apply_chat_template,
#     fn_kwargs = {"tokenizer": tokenizer, "task": "dpo"},
#     num_proc = 12,
#     remove_columns = column_names,
#     desc = "Formatting comparisons with prompt template",
# )

# for split in ["train"]:
#     raw_datasets[split] = raw_datasets[split].rename_columns(
#         {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
#     )

In [3]:
from datasets import load_dataset

# def process_image(example):
#     from qwen_vl_utils import process_vision_info
#     image_inputs, video_inputs = process_vision_info(example["images"])
#     example["images"] = image_inputs
#     return example

dataset = load_dataset("/media/intisar/dataset1/visual_categorization/Sigbath/ds3", split="train")

In [11]:
dataset.features

{'chosen': [{'content': [{'text': Value(dtype='string', id=None),
     'type': Value(dtype='string', id=None)}],
   'role': Value(dtype='string', id=None)}],
 'rejected': [{'content': [{'text': Value(dtype='string', id=None),
     'type': Value(dtype='string', id=None)}],
   'role': Value(dtype='string', id=None)}],
 'prompt': [{'content': [{'text': Value(dtype='string', id=None),
     'type': Value(dtype='string', id=None)}],
   'role': Value(dtype='string', id=None)}],
 'images': Sequence(feature=Image(mode=None, decode=True, id=None), length=-1, id=None)}

In [13]:
model = FastVisionModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 4,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [14]:
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

In [15]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 5e-6,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
    beta = 0.1,
    train_dataset = dataset.select([i for i in range(0, len(dataset), 4)]),
    # eval_dataset = raw_datasets["test"],
    processing_class=tokenizer,
    max_length = 8192,
    max_prompt_length = 2048,
    remove_unused_columns = True,
)

In [10]:
dpo_trainer.train_dataset

Dataset({
    features: ['prompt', 'images', 'prompt_input_ids', 'pixel_values', 'chosen_input_ids', 'rejected_input_ids'],
    num_rows: 1470
})

In [16]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,470 | Num Epochs = 3 | Total steps = 552
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 18,576,384 of 3,773,199,360 (0.49% trained)


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['prompt_input_ids', 'chosen_input_ids', 'rejected_input_ids']